In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms as T
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn as nn
import PIL
from PIL import Image
from torch.optim.lr_scheduler import OneCycleLR
import time
import torchvision.models as models



# modelo UNet

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        
        self.encoder1 = self._block(in_channels, 32)  
        self.encoder2 = self._block(32, 64)         
        self.encoder3 = self._block(64, 128)        

        # Capas de decodificador
        self.upconv3 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder3 = self._block(128, 64)  
        self.upconv2 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.decoder2 = self._block(64, 32)   
        self.decoder1 = self._block(32, out_channels, final_layer=True)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def _block(self, in_channels, features, final_layer=False):
        layers = [
            nn.Conv2d(in_channels, features, kernel_size=3, padding=1),
            nn.BatchNorm2d(features),
            nn.ReLU(inplace=True),
            nn.Conv2d(features, features, kernel_size=3, padding=1),
            nn.BatchNorm2d(features),
            nn.ReLU(inplace=True)
        ]
        if final_layer:
            layers.append(nn.Conv2d(features, out_channels, kernel_size=1))
            layers.append(nn.Sigmoid())
        return nn.Sequential(*layers)

    def forward(self, x):
        # encoder
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool(enc1))
        enc3 = self.encoder3(self.pool(enc2))
        
        # decoder
        dec3 = self.upconv3(enc3)
        dec3 = torch.cat((dec3, enc2), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc1), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.decoder1(dec2)
        
        return dec1
    
# crear el dataset
class CustomDataset(Dataset):
    def __init__(self, image_folder, mask_folder, transform=None):
        self.image_folder = image_folder
        self.mask_folder = mask_folder
        self.transform = transform
        self.images = os.listdir(image_folder)
        self.masks = os.listdir(mask_folder)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.images[idx])
        mask_name = os.path.join(self.mask_folder, self.masks[idx])

    
        image = Image.open(img_name).convert('L') # cambiar depende de los canales
        mask = Image.open(mask_name).convert('L')
        
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask

# transformacion de datos
transform = transforms.Compose([
    transforms.ToTensor(),
])



# dataset y dataloader
custom_dataset = CustomDataset(image_folder='C:/tesis/imagenes/imagenes', mask_folder='C:/tesis/imagenes/mascaras', transform=transform)
train_ratio = 0.8 
train_size = int(train_ratio * len(custom_dataset))
val_size = len(custom_dataset) - train_size
train_dataset, val_dataset = random_split(custom_dataset, [train_size, val_size])


batch_size = 8 # cambiar lotes
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
in_channels = 1  
out_channels = 1  
model = UNet(in_channels, out_channels).to(device=device)
x = torch.randn((1, in_channels, 512, 512)).to(device) # canales y pixeles
predictions = model(x)

# funcion para calcular accuracy
def calculate_accuracy(predictions, masks):
    binary_predictions = (predictions > 0.5).float()
    correct = (binary_predictions == masks).float().sum()
    total = masks.numel()
    accuracy = correct / total
    return accuracy.item()

# Función para calcular la precisión
def calculate_precision(predictions, masks):
    binary_predictions = (predictions > 0.5).float()
    TP = ((binary_predictions == 1) & (masks == 1)).float().sum()
    FP = ((binary_predictions == 1) & (masks == 0)).float().sum()
    precision = TP / (TP + FP + 1e-8)  
    return precision.item()

# entrenamiento del modelo
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses = []
val_losses = []
train_precisions = []
val_precisions = []
train_accuracies = []
val_accuracies = []


num_epochs = 100 # ajustar epocas

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()

    total_train_loss = 0.0
    total_train_accuracy = 0.0
    total_train_precision = 0.0

    # Entrenamiento
    for images, masks in train_loader:
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        accuracy = calculate_accuracy(outputs, masks)
        precision = calculate_precision(outputs, masks)
        total_train_loss += loss.item()
        total_train_accuracy += accuracy
        total_train_precision += precision

    avg_train_loss = total_train_loss / len(train_loader)
    avg_train_accuracy = total_train_accuracy / len(train_loader)
    avg_train_precision = total_train_precision / len(train_loader)

    train_losses.append(avg_train_loss)
    train_precisions.append(avg_train_precision)

    total_val_loss = 0.0
    total_val_accuracy = 0.0
    total_val_precision = 0.0

    # Validación
    model.eval()
    with torch.no_grad():
        for val_images, val_masks in val_loader:
            val_images, val_masks = val_images.to(device), val_masks.to(device)
            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, val_masks)

            accuracy = calculate_accuracy(val_outputs, val_masks)
            precision = calculate_precision(val_outputs, val_masks)
            total_val_loss += val_loss.item()
            total_val_accuracy += accuracy
            total_val_precision += precision

    avg_val_loss = total_val_loss / len(val_loader)
    avg_val_accuracy = total_val_accuracy / len(val_loader)
    avg_val_precision = total_val_precision / len(val_loader)
    
    val_losses.append(avg_val_loss)
    val_precisions.append(avg_val_precision)

    end_time = time.time()
    epoch_time = end_time - start_time

    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Loss: {avg_train_loss:.4f}, '
          f'Prec: {avg_train_precision:.4f}, '
          f'Acc: {avg_train_accuracy:.4f}, '
          f'Val_Loss: {avg_val_loss:.4f}, '
          f'Val_Prec: {avg_val_precision:.4f}, '
          f'Val_Acc: {avg_val_accuracy:.4f}, '
          f'Time: {epoch_time:.2f} sec')


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.plot(train_losses, label='Pérdida de Entrenamiento')
plt.plot(val_losses, label='Pérdida de Validación')
plt.title('Función de Pérdida durante el Entrenamiento y la Validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(train_precisions, label='Precisión de Entrenamiento')
plt.plot(val_precisions, label='Precisión de Validación')
plt.title('Precisión durante el Entrenamiento y la Validación')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.show()

In [ ]:
def calculate_sensitivity(TP, FN):
    return TP / (TP + FN + 1e-8)

def calculate_specificity(TN, FP):
    return TN / (TN + FP + 1e-8)

def calculate_f1_score(precision, recall):
    return 2 * (precision * recall) / (precision + recall + 1e-8)

def calculate_confusion_matrix(predictions, masks):
    binary_predictions = (predictions > 0.5).float()
    TP = ((binary_predictions == 1) & (masks == 1)).float().sum().item()
    FP = ((binary_predictions == 1) & (masks == 0)).float().sum().item()
    TN = ((binary_predictions == 0) & (masks == 0)).float().sum().item()
    FN = ((binary_predictions == 0) & (masks == 1)).float().sum().item()
    return TP, FP, TN, FN

def calculate_precision(TP, FP):
    precision = TP / (TP + FP + 1e-8)  # Añadido 1e-8 para evitar división por cero
    return precision

total_val_TP = 0.0
total_val_FP = 0.0
total_val_TN = 0.0
total_val_FN = 0.0

# Validación
model.eval()
with torch.no_grad():
    for val_images, val_masks in val_loader:
        val_images, val_masks = val_images.to(device), val_masks.to(device)
        val_outputs = model(val_images)

        TP, FP, TN, FN = calculate_confusion_matrix(val_outputs, val_masks)
        total_val_TP += TP
        total_val_FP += FP
        total_val_TN += TN
        total_val_FN += FN

# Calcula las métricas finales
val_precision = calculate_precision(total_val_TP, total_val_FP)
val_recall = calculate_sensitivity(total_val_TP, total_val_FN)  # Sensibilidad
val_specificity = calculate_specificity(total_val_TN, total_val_FP)
val_f1_score = calculate_f1_score(val_precision, val_recall)

print(f'Precision: {val_precision:.4f}, Sensibilidad: {val_recall:.4f}, '
      f'Especificidad: {val_specificity:.4f}, F1-Score: {val_f1_score:.4f}')
print(f'Matriz de Confusión: Verdaderos Positivos={total_val_TP}, Falsos Positivos={total_val_FP}, Verdaderos Negativos={total_val_TN}, Falsos Negativos={total_val_FN}')